In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.4 MB/s eta 0:00:00


In [ ]:
import os
import yaml

# Đường dẫn folder dataset trên Colab (thường là cái này nếu bạn upload trực tiếp)
dataset_root = '/content/drive/MyDrive/Processed_Dataset'

# Nội dung file yaml chuẩn
data_config = {
    'path': os.path.abspath(dataset_root),  # Quan trọng: Lấy đường dẫn tuyệt đối
    'train': 'train/images',
    'val': 'val/images',
    'nc': 3,
    'names': {
        0: 'Fall Detected',
        1: 'Walking',
        2: 'Sitting'
    }
}

# Ghi đè lại file data.yaml nằm bên trong folder dataset
yaml_path = os.path.join(dataset_root, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"Đã cập nhật file cấu hình tại: {yaml_path}")
print(f"Đường dẫn gốc (path) mới là: {data_config['path']}")

Đã cập nhật file cấu hình tại: /content/drive/MyDrive/Processed_Dataset/data.yaml
Đường dẫn gốc (path) mới là: /content/drive/MyDrive/Processed_Dataset


In [ ]:
import os
import shutil
import pandas as pd
from ultralytics import YOLO, RTDETR
from google.colab import drive

MANUAL_CONFIG = {
    'data': '/content/drive/MyDrive/Processed_Dataset/data.yaml',
    'epochs': 60,             
    'imgsz': 640,
    'batch': 16,
    'patience': 20,           
    'close_mosaic': 10,       
    'save': True,
    'verbose': True,
    'exist_ok': True
}

# 3. Danh sách 3 mô hình để so sánh
models_list = [
    {'file': 'yolov8n.pt',  'name': 'YOLOv8n',  'type': 'YOLO'},
    {'file': 'yolov10n.pt', 'name': 'YOLOv10n', 'type': 'YOLO'},
    {'file': 'rtdetr-l.pt', 'name': 'RTDETR',   'type': 'RTDETR'}
]

results_data = []

# 4. Chạy vòng lặp
for m in models_list:
    print(f"\n>>> ĐANG CHẠY: {m['name']} ...")

    if m['type'] == 'YOLO':
        model = YOLO(m['file'])
    else:
        model = RTDETR(m['file'])

    # Truyền bộ tham số thủ công vào
    model.train(project='/content/runs/final_report', name=m['name'], **MANUAL_CONFIG)

    # Lấy kết quả để so sánh
    csv_path = f"/content/runs/final_report/{m['name']}/results.csv"
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df.columns = [c.strip() for c in df.columns]
        results_data.append({
            'Model': m['name'],
            'mAP@50': df['metrics/mAP50(B)'].max(),
            'mAP@50-95': df['metrics/mAP50-95(B)'].max(),
            'Params': MANUAL_CONFIG 
        })

# 5. Xuất bảng so sánh
df_res = pd.DataFrame(results_data)
print("\n=== KẾT QUẢ SO SÁNH CUỐI CÙNG (CÙNG CẤU HÌNH THỦ CÔNG) ===")
print(df_res[['Model', 'mAP@50', 'mAP@50-95']])

# 6. Lưu về Drive
shutil.copytree('/content/runs/final_report', '/content/drive/MyDrive/Final_Report_Results', dirs_exist_ok=True)


>>> ĐANG CHẠY: YOLOv8n ...
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Processed_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/60      12.8G     0.6147      17.72     0.6602         30        640: 100% ━━━━━━━━━━━━ 23/23 1.2s/it 28.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.0it/s 2.0s
                   all        110        113     0.0464      0.193     0.0853     0.0516

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/60      12.6G     0.3784      1.354      0.379         27        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.7s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.8it/s 2.3s
                   all        110        113      0.839      0.272      0.225      0.135

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      59/60      12.6G     0.1587     0.2749     0.1978         13        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.0it/s 2.0s
                   all        110        113      0.808      0.821      0.804      0.573

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      60/60      12.6G     0.1641     0.2833      0.217         11        640: 100% ━━━━━━━━━━━━ 23/23 1.1s/it 25.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.9it/s 2.1s
                   all        110        113      0.792      0.837      0.802      0.573

60 epochs completed in 0.688 hours.
Optimizer stripped from /content/runs/final_report/RTDETR/weights/last.pt, 66.2MB
Optimizer stripped from /content/runs/final_report/RTDETR/weights/best.pt, 66.2MB

Validating /content/runs/final_report/RTDETR/weights/best.pt...
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
rt-detr-l summary: 310 layers, 31,989,905 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.3it/s 3.0s
                   all        110        113      0.809      0.801      0.805       0.58
         Fall Detected    

'/content/drive/MyDrive/Final_Report_Results'